In [3]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers import BatchNormalization, LeakyReLU
from keras.optimizers import Adam
from keras.datasets import mnist
import tensorflow as tf

# Dimensões das imagens e do ruído
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)

# Função para construir o gerador da GAN
def build_generator():
    noise_shape = 100

    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(1024, input_dim=noise_shape, activation='relu'))
    model.add(tf.keras.layers.Dense(784, activation='tanh'))
    model.add(tf.keras.layers.Reshape((28, 28, 1)))
    model.summary()

    noise = Input(shape=noise_shape)
    img = model(noise)  # Imagem gerada
    return Model(noise, img)

# Função para construir o discriminador da GAN
def build_discriminator():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=img_shape))
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    model.summary()

    img = Input(shape=img_shape)
    validity = model(img)
    return Model(img, validity)

# Função principal para treinar a GAN
def train(epochs, batch_size=128, save_interval=50):
    save1 = ''
    # Carrega o conjunto de dados MNIST
    (X_train, _), (_, _) = mnist.load_data()
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5
    X_train = np.expand_dims(X_train, axis=3) 
    half_batch = int(batch_size / 2)
    
    for epoch in range(epochs):
        # Treina o discriminador em lotes de imagens reais
        idx = np.random.randint(0, X_train.shape[0], half_batch)
        imgs = X_train[idx]
        noise = np.random.normal(0, 1, (half_batch, 100))
        gen_imgs = generator.predict(noise)
        d_loss_real = discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
        
        # Treina o discriminador em lotes de imagens geradas
        d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake) 
        
        noise = np.random.normal(0, 1, (batch_size, 100)) 
        valid_y = np.array([1] * batch_size) 
        
        # Treina o gerador através da GAN combinada
        g_loss = combined.train_on_batch(noise, valid_y)
        
        # Limpa a saída para melhor visualização
        from IPython.display import clear_output
        clear_output(wait=True)
        print(save1)
        print('\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n')
        
        # Salva imagens geradas em intervalos específicos
        if epoch % 100 == 0:
            save1 += ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]\n" % (epoch, d_loss[0], 100 * d_loss[1], g_loss))
            save_imgs(epoch)
        
        # Salva o modelo do gerador em intervalos específicos
        if epoch % save_interval == 0:
            generator.save(f'models/generator_{epoch}_MNIST_SIMPLES.h5')

# Função para salvar imagens geradas
def save_imgs(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, 100))
    gen_imgs = generator.predict(noise)

    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images_simples/s_mnist_%d.png" % epoch)
    plt.close()

# Otimizador Adam
optimizer = Adam(0.0002, 0.5)  

# Constrói e compila o discriminador
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy',
                      optimizer=optimizer,
                      metrics=['accuracy'])

# Constrói e compila o gerador
generator = build_generator()
generator.compile(loss='binary_crossentropy', optimizer=optimizer)

# Entrada e saída para a GAN combinada
z = Input(shape=(100,))  
img = generator(z)
discriminator.trainable = False
valid = discriminator(img)
combined = Model(z, valid)
combined.compile(loss='binary_crossentropy', optimizer='adam')  

# Treina a GAN
train(epochs=5000, batch_size=32, save_interval=1000)

# Salva o modelo do gerador após o treinamento
generator.save('generator_model.h5')


0 [D loss: 0.505230, acc.: 62.50%] [G loss: 0.662298]
100 [D loss: 0.028185, acc.: 100.00%] [G loss: 3.202781]
200 [D loss: 0.048329, acc.: 100.00%] [G loss: 3.376809]
300 [D loss: 0.064822, acc.: 100.00%] [G loss: 3.008972]
400 [D loss: 0.168100, acc.: 96.88%] [G loss: 1.560899]
500 [D loss: 0.037421, acc.: 100.00%] [G loss: 4.154796]
600 [D loss: 0.020553, acc.: 100.00%] [G loss: 3.595398]
700 [D loss: 0.041561, acc.: 100.00%] [G loss: 4.948656]
800 [D loss: 0.017606, acc.: 100.00%] [G loss: 4.313765]
900 [D loss: 0.008132, acc.: 100.00%] [G loss: 6.054270]
1000 [D loss: 0.006055, acc.: 100.00%] [G loss: 5.916219]
1100 [D loss: 0.011224, acc.: 100.00%] [G loss: 6.565580]
1200 [D loss: 0.006912, acc.: 100.00%] [G loss: 6.564747]
1300 [D loss: 0.001115, acc.: 100.00%] [G loss: 7.182378]
1400 [D loss: 0.005855, acc.: 100.00%] [G loss: 6.901257]
1500 [D loss: 0.001379, acc.: 100.00%] [G loss: 7.106753]
1600 [D loss: 0.001441, acc.: 100.00%] [G loss: 6.853991]
1700 [D loss: 0.001745, acc.